# Google Bigtable

> [Google Cloud Bigtable](https://cloud.google.com/bigtable) 是一种键值和宽列存储，非常适合对结构化、半结构化或非结构化数据进行快速访问。通过 Bigtable 的 Langchain 集成，扩展您的数据库应用程序以构建支持 AI 的体验。

本笔记本将介绍如何使用 `BigtableChatMessageHistory` 类通过 [Google Cloud Bigtable](https://cloud.google.com/bigtable) 存储聊天消息历史记录。

在 [GitHub](https://github.com/googleapis/langchain-google-bigtable-python/) 上了解有关该软件包的更多信息。

[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-bigtable-python/blob/main/docs/chat_message_history.ipynb)

## 开始之前

要运行此笔记本，您需要执行以下操作：

* [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
* [启用 Bigtable API](https://console.cloud.google.com/flows/enableapi?apiid=bigtable.googleapis.com)
* [创建 Bigtable 实例](https://cloud.google.com/bigtable/docs/creating-instance)
* [创建 Bigtable 表](https://cloud.google.com/bigtable/docs/managing-tables)
* [创建 Bigtable 访问凭据](https://developers.google.com/workspace/guides/create-credentials)

### 🦜🔗 库安装

该集成位于独立的 `langchain-google-bigtable` 包中，因此我们需要安装它。

In [ ]:
%pip install -upgrade --quiet langchain-google-bigtable

**仅限 Colab**：取消注释下面的单元格以重启内核，或者使用此按钮重启内核。对于 Vertex AI Workbench，您可以重启顶部的终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便在本笔记本中利用 Google Cloud 资源。

如果您不知道项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 🔐 身份验证

通过笔记本中登录的 IAM 用户身份验证到 Google Cloud，以便访问您的 Google Cloud 项目。

- 如果您正在使用 Colab 运行此笔记本，请使用下面的单元格并继续。
- 如果您正在使用 Vertex AI Workbench，请查看此处的设置说明：[https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [ ]:
from google.colab import auth

auth.authenticate_user()

## 基本用法

### 初始化 Bigtable 架构

`BigtableChatMessageHistory` 的架构要求实例和表已存在，并且有一个名为 `langchain` 的列族。

In [ ]:
# @markdown Please specify an instance and a table for demo purpose.
INSTANCE_ID = "my_instance"  # @param {type:"string"}
TABLE_ID = "my_table"  # @param {type:"string"}

如果表或列族不存在，您可以使用以下函数来创建它们：

In [ ]:
from google.cloud import bigtable
from langchain_google_bigtable import create_chat_history_table

create_chat_history_table(
    instance_id=INSTANCE_ID,
    table_id=TABLE_ID,
)

### BigtableChatMessageHistory

初始化 `BigtableChatMessageHistory` 类只需要提供以下 3 项：

1.  `instance_id` - 用于聊天消息历史记录的 Bigtable 实例。
2.  `table_id` - 用于存储聊天消息历史记录的 Bigtable 表。
3.  `session_id` - 指定会话 ID 的唯一标识符字符串。

In [ ]:
from langchain_google_bigtable import BigtableChatMessageHistory

message_history = BigtableChatMessageHistory(
    instance_id=INSTANCE_ID,
    table_id=TABLE_ID,
    session_id="user-session-id",
)

message_history.add_user_message("hi!")
message_history.add_ai_message("whats up?")

In [ ]:
message_history.messages

#### 清理

当特定会话的历史记录不再需要且可以删除时，可以按以下方式进行。

**注意：** 一旦删除，数据将不再存储在 Bigtable 中，并且将永久消失。

In [ ]:
message_history.clear()

## 高级用法

### 自定义客户端
默认创建的客户端是默认客户端，仅使用 `admin=True` 选项。要使用非默认客户端，可以将自定义客户端传递给构造函数。

In [ ]:
from google.cloud import bigtable

client = (bigtable.Client(...),)

create_chat_history_table(
    instance_id="my-instance",
    table_id="my-table",
    client=client,
)

custom_client_message_history = BigtableChatMessageHistory(
    instance_id="my-instance",
    table_id="my-table",
    client=client,
)